In [1]:
import pandas as pd
import os
import csv
import re
import logging
import optparse
import re
import spacy
import dedupe
import pickle
import copy
import json
from unidecode import unidecode

In [2]:
sp = spacy.load('en_core_web_sm')

In [3]:
def format_number(num):
    num = float(num)
    if num % 1 == 0:
        return int(num)
    else:
        return num
def fill_nulls_with_none(df):
    """ Fills nulls in a dataframe with None.
        This is required for the Dedupe package to work properly.

        Input: - dataframe with nulls as NaN

        Output: - new dataframe with nulls as None
    """
    new_df = df.copy()
    for col in df.columns:
        new_df[col] = new_df[col].where(new_df[col].notnull(), None)
    return new_df

def convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):
    """ Convert number types to strings in a dataframe.
        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!

        Inputs: - df -> dataframe to convert number types
                - cols_to_convert -> list of columns to convert
                - remove_point_zero -> bool to say whether you want '.0' removed from number

        Outputs: - dataframe with converted number types
    """
    new_df = df.copy()
    for col in cols_to_convert:
        if remove_point_zero:
            new_df[col] = new_df[col].apply(lambda x: str(x).replace('.0','')\
                                            if not isinstance(x, type(None)) else x)
        else:
            new_df[col] = new_df[col].apply(lambda x: str(x)\
                                            if not isinstance(x, type(None)) else x)
    return new_df

extra_brands = set(pd.read_csv('laptops.csv').Company.str.lower().unique())
screen_sizes = set(pd.read_csv('laptops.csv').Inches)
screen_sizes = [str(format_number(str(s).lower())) for s in screen_sizes]


In [4]:
# From Jerry's code. Might not be needed:
title_remove_words = ["price", "comparison", "at", "buy.net", "amazon.com", ":",
"computers", "&", "accessories", "laptop", "vology", "tigerdirect", ".com", "ultraportable", "cool",
"audiophile", "wireless", "bluetooth", "speaker", "portable", "with", "built-in", "microphone", "and",
"micro", "sd","card","slot", "-", "(", ")", "high", "performance", "new", "core", "high", "end", "bes",
"audio", "nx.m8eaa.007", "/", "notebook", "pc", '"', "brand", "new", "hewlett-packard"]

def preprocess_laptop_dataset(df):
    # Alpha numeric
    irrelevant_regex = re.compile(r"[^a-z0-9,.\-\s]")
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
    df.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

    for column in df.columns:
        if column == 'instance_id':
            continue
        df[column] = df[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')


    def tokenize_new_tile(record):
        return [w.text for w in sp(record['new_title'])]

    df['new_title'] = df.title
    irrelevant_regex = re.compile(r"[^a-z0-9.\s]")
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
    df['new_title'] = df.new_title.str.lower().str.replace(irrelevant_regex, '').str.replace(multispace_regex, ' ')
    df['new_title_tokens'] =  df.apply(tokenize_new_tile, axis=1)

    # Brand assignment
    all_brands = set()

    all_brands.update(extra_brands)

    def assign_brand(record):
        # Search in brand first
        if record['brand'] in all_brands:
            return record['brand']
        # then in the title
        for el in all_brands:
            if el in record['title']:
                return el
        return "NNN"

    df['brand'] = df.apply(assign_brand, axis=1)

    # cpu brand
    intel=['intel', 'i3', 'i5', 'i7'] #Needed because not all entries have intel
    def assign_cpu_brand(record):
        # Search in brand first
        for blue in intel:
            if blue in str(record['cpu_brand']) or blue in str(record['title']) or \
                    blue in str(record['cpu_model']) or blue in str(record['cpu_type']):
                return 'intel'
        return 'amd'

    df['cpu_brand'] = df.apply(assign_cpu_brand, axis=1)

    def assign_screen_size(record):
        brand_tokens = record['new_title_tokens']
        arr = []
        for t in brand_tokens:
            s = t.replace('inch', '')
            s = s.replace('in', '')
            arr.append(s)

        for sc in screen_sizes:
            if str(sc) in arr:
                return str(sc)

        else:
            return str(15.6) # Some relaxation
    df['screen_size'] = df.apply(assign_screen_size, axis=1)


    # # ram capacity
    # def assign_ram_capacity(record):
    #     s = str(record['ram_capacity']).replace(' ', '')
    #     possible_vals = ['2gb', '4gb', '6gb', '8gb', '10gb', '12gb', '16gb',
    #                      '32gb', '64gb', '128gb', '256gb', '512gb', '2', '4',
    #                      '6', '8', '10', '12', '16', '32', '64', '128']
    #     for val in possible_vals:
    #         if val in s:
    #             return int(val.replace('gb', ''))
    #
    #     s = str(record['title']).replace(' ', '')  # This will be wrong, please change
    #     possible_vals = ['2gb', '4gb', '6gb', '8gb', '10gb', '12gb', '16gb',
    #                      '32gb', '64gb', '128gb']
    #     for val in possible_vals:
    #         if val in s:
    #             return int(val.replace('gb', ''))
    #
    #     return 0
        #new ram capacity
    def assign_ram_capacity(record):
        s = str(record['ram_capacity'])
        t = str(record['title'])
        regex = re.compile(r"(\d{1,3})\s?([gm]b)") # rare chance of encountering MB as an error
        m = None
        #ram_c = df['ram_capacity'].str.extract(regex)
        #title_ram = df['title'].str.extract(regex)
        if s:
            m=re.search(regex, s)
        if m is None:
            m=re.search(regex, t)
        if m is None:
            return None
        else:
            m= m.group()
            return re.sub(r'([gm]b)', "", m) # remove MB and GB

    df['ram_capacity'] = df.apply(assign_ram_capacity, axis=1)

    def assign_ram_type(record):
        m = None
        if "ddr3" in record['ram_type']:
            return "ddr3"

    df['ram_type'] = df.apply(assign_ram_type, axis=1)


    def assign_hdd_capacity(record):
        s = str(record['hdd_capacity']).replace(' ', '')
        s2 = str(record['title'].replace(' ', ''))

        if 'ssd' in s:
            return 0

        if re.search("\d{3,4}gb", s):
            return int(re.findall("\d{3,4}gb", s)[0][:-2])
        if re.search("\dtb", s):
            return int(re.findall("\dtb", s)[0][:-2] + '000')
        if re.search("\d{3,4}gbhdd", s2):
            return int(re.findall("\d{3,4}gbhdd", s2)[0][:-5])
        if re.search("hdd\d{3,4}gb", s2):
            return int(re.findall("hdd\d{3,4}gb", s2)[0][3:-2])
        if re.search("hdd\dtb", s2):
            return int(re.findall("hdd\dtb", s2)[0][3:4] + '000')
        if re.search("\dtbhdd", s2):
            return int(re.findall("\dtbhdd", s2)[0][0] + '000')
        return 0
    df['hdd_capacity'] = df.apply(assign_hdd_capacity, axis=1)

    def assign_ssd_capacity(record):
        s = str(record['ssd_capacity']).replace(' ', '')
        s2 = str(record['title'].replace(' ', ''))


        if re.search("\d{3,4}gb", s):
            return int(re.findall("\d{3,4}gb", s)[0][:-2])
        if re.search("\dtb", s):
            return int(re.findall("\dtb", s)[0][:-2] + '000')
        if re.search("\d{3,4}gbssd", s2):
            return int(re.findall("\d{3,4}gbssd", s2)[0][:-5])
        if re.search("ssd\d{3,4}gb", s2):
            return int(re.findall("ssd\d{3,4}gb", s2)[0][3:-2])
        if re.search("ssd\dtb", s2):
            return int(re.findall("ssd\dtb", s2)[0][3:4] + '000')
        if re.search("\dtbssd", s2):
            return int(re.findall("\dtbssd", s2)[0][0] + '000')
        return 0

    df['ssd_capacity'] = df.apply(assign_ssd_capacity, axis=1)

    def assign_laptop_model(record):
        brand_tokens = record['new_title_tokens']
        try:
            brand_index = brand_tokens.index(str(record['brand']))
            finish_index = brand_index + 2
            should_break = False
            for i in range(2 + brand_index, 5 + brand_index, 1):
                for sc in screen_sizes:
                    if sc in brand_tokens[i]:
                        should_break = True
                        break
                if should_break:
                    if finish_index == i:
                        finish_index -=1
                    break
                if not (brand_tokens[i].isalpha()):
                    finish_index = i
                else:
                    break
        except:
            brand_index = -1

        if brand_index == -1:
            return None

        return ' '.join(brand_tokens[brand_index+1:finish_index+1])

    df['model'] = df.apply(assign_laptop_model, axis=1)


    df = fill_nulls_with_none(df)
    df = convert_numbers_to_strings(df, ['screen_size'])
    # Unit stand. in weight
    def assign_weight(record): #TO DO: Convert kg to lb if needed
        regex=re.compile('.?(\d{1,2}\.\d{1,2})\s?[lpk]')
        s = record['weight']
        m = None
        if s:
            m = re.search(regex, s)
        if m is None:
            m = re.search(regex, record['title'])
        if m is None:
            return None
        else:
            m = m.group()
            return re.sub(r"\s?[lpk]", "", m)

    df['weight'] = df.apply(assign_weight, axis=1)

    def assign_cpu_type(record):
    # Find the cpu type
        cpu_list = ["i5", "i3", "i7", "atom",
                    "pentium", "celeron", "a-series",
                    "e-series", "aseries", "eseries",
                    "a1", "a2", "a3", "a4", "a5", "a6", "a7", "a8", "a9"]

        for cpu in cpu_list:
            if record['cpu_type'] is not None and cpu in record['cpu_type']:
                return cpu
            if cpu in record['title']:
                return cpu
            if record['cpu_model'] is not None and cpu in record['cpu_model']:
                return cpu
            if record['cpu_frequency'] is not None and  cpu in record['cpu_frequency']:
                return cpu

            if re.search("e-[0-9]{3}", record['title']):
                return re.findall("e-[0-9]{3}", record['title'])[0]

            if record['cpu_model'] is not None and re.search("e-[0-9]{3}", record['cpu_model']):
                return re.findall("e-[0-9]{3}", record['cpu_model'])[0]

    df['cpu_type'] = df.apply(assign_cpu_type, axis=1)

    #TO DO: there are laptops called E1-572 and cpus called E1-2100 or E-300
    def assign_cpu_model(record):
        model=record['cpu_model']
        regex=re.compile(r"-?\d{1,4}([mu])") #For intel cpus
        regex2=re.compile(r"[ea]\d?-\d{1,4}[m]?") #for amd A and E series. Needs detection after AMD tag in title
        m=None
        if record['cpu_brand']=='intel' and model is not None :
            m = re.search(regex, model)
            if m is not None:
                m=m.group()
                return re.sub(r'-', "", m)
        if re.search("intel", record['title']): # one case where laptop model is 50m and gets caught
            m = re.search(regex, record['title'])
            if m is not None:
                m=m.group()
                return re.sub(r'-', "", m)
        if record['cpu_brand']=='amd' and model is not None:
            m = re.search(regex2, model)
            if m is not None:
                m=m.group()
                return re.sub(r'[ea]\d?-', "", m)
        if re.search("amd", record['title']):
            m = re.search(regex2, record['title'])
            if m is not None:
                m=m.group()
                return re.sub(r'[ea]\d?-', "", m)
        if m is None:
            return None

    df['cpu_model'] = df.apply(assign_cpu_model, axis=1)

    def assign_cpu_frequency(record):
        s = record['cpu_frequency']
        regex=re.compile(r"\d?.\d{1,2}\s?ghz")
        m = None
        if s:
            m=re.search(regex, s)
            if m is not None:
                m=m.group()
                return re.sub(r'ghz', "", m)
        if re.search("ghz", record['title']):
            m = re.search(regex, record['title'])
            if m is not None:
                m=m.group()
                return re.sub(r'ghz', "", m)
        if m is None:
            return None
    df['cpu_frequency'] = df.apply(assign_cpu_frequency, axis=1)

    return df

In [18]:
'''
Redundant with previous functions
def fill_nulls_with_none(df):
    """ Fills nulls in a dataframe with None.
        This is required for the Dedupe package to work properly.
        
        Input: - dataframe with nulls as NaN
        
        Output: - new dataframe with nulls as None
    """
    new_df = df.copy()
    for col in df.columns:
        new_df[col] = new_df[col].where(new_df[col].notnull(), None)
    return new_df

def convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):
    """ Convert number types to strings in a dataframe.
        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!
        
        Inputs: - df -> dataframe to convert number types
                - cols_to_convert -> list of columns to convert
                - remove_point_zero -> bool to say whether you want '.0' removed from number
        
        Outputs: - dataframe with converted number types
    """
    new_df = df.copy()
    for col in cols_to_convert:
        if remove_point_zero:
            new_df[col] = new_df[col].apply(lambda x: str(x).replace('.0','')\
                                            if not isinstance(x, type(None)) else x)
        else:
            new_df[col] = new_df[col].apply(lambda x: str(x)\
                                            if not isinstance(x, type(None)) else x)
    return new_df
'''

' Redundant with previous functions\ndef fill_nulls_with_none(df):\n    """ Fills nulls in a dataframe with None.\n        This is required for the Dedupe package to work properly.\n        \n        Input: - dataframe with nulls as NaN\n        \n        Output: - new dataframe with nulls as None\n    """\n    new_df = df.copy()\n    for col in df.columns:\n        new_df[col] = new_df[col].where(new_df[col].notnull(), None)\n    return new_df\n\ndef convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):\n    """ Convert number types to strings in a dataframe.\n        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!\n        \n        Inputs: - df -> dataframe to convert number types\n                - cols_to_convert -> list of columns to convert\n                - remove_point_zero -> bool to say whether you want \'.0\' removed from number\n        \n        Outputs: - dataframe with converted number types\n    """\n    new_df = df.co

In [5]:
x2 = pd.read_csv("../data/sigmod/X2.csv")
x2.set_index('instance_id',  inplace=True, drop=False)
x2 = preprocess_laptop_dataset(x2.copy(deep=True))
#x2.ram_capacity[50:60]

In [6]:
x3 = pd.read_csv("../data/sigmod/X3.csv")
x3.set_index('instance_id',  inplace=True, drop=False)
x3 = preprocess_laptop_dataset(x3.copy(deep=True))
# x3 = convert_numbers_to_strings(x3, ['ram_capacity', 'screen_size'])

In [7]:
len(x2), len(x3)

(343, 337)

In [104]:
#x3.head()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,new_title,new_title_tokens,screen_size,model
instance_id,,,,,,,,,,,,,,,,,,
www.softwarecity.ca//737,www.softwarecity.ca//737,lenovo,intel,i5-3320m,i5,2.60 ghz,4,ddr3 sdram. ddr3-1600 pc3-12800. ddr3 sdram,ddr3-1600 pc3-12800,320,0,1.80 kg,None,lenovo thinkpad x230 34352jf tablet pc - 12.5 ...,lenovo thinkpad x230 34352jf tablet pc 12.5 in...,"[lenovo, thinkpad, x230, 34352jf, tablet, pc, ...",12.5,thinkpad x230 34352jf
www.isupplyhub.com//1256,www.isupplyhub.com//1256,acer,intel,None,i5,1.6 ghz intel core i5-4200u,8,ddr3 sdram. 8 gb ddr3l sdram,None,500,0,4.8 pounds,15.02 x 10.08 x 0.90 inches,amazon.com acer aspire v7-582pg-6479 15.6-inch...,amazon.com acer aspire v7582pg6479 15.6inch to...,"[amazon.com, acer, aspire, v7582pg6479, 15.6in...",15.6,aspire v7582pg6479
www.isupplyhub.com//326,www.isupplyhub.com//326,acer,intel,None,i5,1.6 ghz intel core i5,4,ddr3 sdram. 4 gb ddr3-sdram,None,500,0,5.2 pounds,15.02 x 10.08 x 1 inches,amazon.com acer aspire e1-572-6870 15.6 inch l...,amazon.com acer aspire e15726870 15.6 inch lap...,"[amazon.com, acer, aspire, e15726870, 15.6, in...",15.6,aspire
www.isupplyhub.com//821,www.isupplyhub.com//821,hp,amd,None,None,None,4,ddr3 sdram. 4 gb sdram ddr3,None,500,0,4.8 pounds,15.18 x 0.89 x 10.16 inches,amazon.com 15.6 hp 15-f009wm amd dual-core e1-...,amazon.com 15.6 hp 15f009wm amd dualcore e1210...,"[amazon.com, 15.6, hp, 15f009wm, amd, dualcore...",15.6,15f009wm amd
www.isupplyhub.com//157,www.isupplyhub.com//157,asus,intel,None,i5,1.7 ghz core i5-3317u,4,ddr3 sdram. 4 gb ddr3,None,0,256,2.9 pounds,8.80 x 0.70 x 12.80 inches,amazon.com asus ux31a-xb52 13.3-inch ultrabook...,amazon.com asus ux31axb52 13.3inch ultrabook 1...,"[amazon.com, asus, ux31axb52, 13.3inch, ultrab...",13.3,ux31axb52


In [8]:
x2.index.intersection(x3.index)

Index([], dtype='object', name='instance_id')

In [9]:
# x23 = x3.append(x2)
x23 = x2

In [10]:
x23.head()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,new_title,new_title_tokens,screen_size,model
instance_id,,,,,,,,,,,,,,,,,,
www.softwarecity.ca//737,www.softwarecity.ca//737,lenovo,intel,3320m,i5,2.60,4,ddr3,ddr3-1600 pc3-12800,320,0,1.80,None,lenovo thinkpad x230 34352jf tablet pc - 12.5 ...,lenovo thinkpad x230 34352jf tablet pc 12.5 in...,"[lenovo, thinkpad, x230, 34352jf, tablet, pc, ...",12.5,thinkpad x230 34352jf
www.isupplyhub.com//1256,www.isupplyhub.com//1256,acer,intel,None,i5,1.6,8,ddr3,None,500,0,4.8,15.02 x 10.08 x 0.90 inches,amazon.com acer aspire v7-582pg-6479 15.6-inch...,amazon.com acer aspire v7582pg6479 15.6inch to...,"[amazon.com, acer, aspire, v7582pg6479, 15.6in...",15.6,aspire v7582pg6479
www.isupplyhub.com//326,www.isupplyhub.com//326,acer,intel,4200u,i5,1.6,4,ddr3,None,500,0,5.2,15.02 x 10.08 x 1 inches,amazon.com acer aspire e1-572-6870 15.6 inch l...,amazon.com acer aspire e15726870 15.6 inch lap...,"[amazon.com, acer, aspire, e15726870, 15.6, in...",15.6,aspire
www.isupplyhub.com//821,www.isupplyhub.com//821,hp,amd,2100,None,None,4,ddr3,None,500,0,4.8,15.18 x 0.89 x 10.16 inches,amazon.com 15.6 hp 15-f009wm amd dual-core e1-...,amazon.com 15.6 hp 15f009wm amd dualcore e1210...,"[amazon.com, 15.6, hp, 15f009wm, amd, dualcore...",15.6,15f009wm amd
www.isupplyhub.com//157,www.isupplyhub.com//157,asus,intel,3317u,i5,1.7,4,ddr3,None,0,256,2.9,8.80 x 0.70 x 12.80 inches,amazon.com asus ux31a-xb52 13.3-inch ultrabook...,amazon.com asus ux31axb52 13.3inch ultrabook 1...,"[amazon.com, asus, ux31axb52, 13.3inch, ultrab...",13.3,ux31axb52


In [11]:
x23.columns, len(x23.columns)

(Index(['instance_id', 'brand', 'cpu_brand', 'cpu_model', 'cpu_type',
        'cpu_frequency', 'ram_capacity', 'ram_type', 'ram_frequency',
        'hdd_capacity', 'ssd_capacity', 'weight', 'dimensions', 'title',
        'new_title', 'new_title_tokens', 'screen_size', 'model'],
       dtype='object'),
 18)

In [12]:
to_dedupe = x23[[
    'instance_id',
    'brand',
    'cpu_brand',
    'cpu_type',
    'ram_capacity',
    'hdd_capacity', 
    'ssd_capacity',
    'title',
    'screen_size',
    'model']].copy()

In [13]:
to_dedupe.head()

,instance_id,brand,cpu_brand,cpu_type,ram_capacity,hdd_capacity,ssd_capacity,title,screen_size,model
instance_id,,,,,,,,,,
www.softwarecity.ca//737,www.softwarecity.ca//737,lenovo,intel,i5,4,320,0,lenovo thinkpad x230 34352jf tablet pc - 12.5 ...,12.5,thinkpad x230 34352jf
www.isupplyhub.com//1256,www.isupplyhub.com//1256,acer,intel,i5,8,500,0,amazon.com acer aspire v7-582pg-6479 15.6-inch...,15.6,aspire v7582pg6479
www.isupplyhub.com//326,www.isupplyhub.com//326,acer,intel,i5,4,500,0,amazon.com acer aspire e1-572-6870 15.6 inch l...,15.6,aspire
www.isupplyhub.com//821,www.isupplyhub.com//821,hp,amd,None,4,500,0,amazon.com 15.6 hp 15-f009wm amd dual-core e1-...,15.6,15f009wm amd
www.isupplyhub.com//157,www.isupplyhub.com//157,asus,intel,i5,4,0,256,amazon.com asus ux31a-xb52 13.3-inch ultrabook...,13.3,ux31axb52


In [14]:
to_dedupe_dict = to_dedupe.to_dict(orient = 'index')

In [15]:
to_dedupe_dict['www.softwarecity.ca//737']

{'instance_id': 'www.softwarecity.ca//737',
 'brand': 'lenovo',
 'cpu_brand': 'intel',
 'cpu_type': 'i5',
 'ram_capacity': '4 ',
 'hdd_capacity': 320,
 'ssd_capacity': 0,
 'title': 'lenovo thinkpad x230 34352jf tablet pc - 12.5 - in-plane switching ips technology - wireless lan - intel core i5 i5-3320m 2.60 ghz - black - 4 gb ram - 320 gb hdd - windows 7 professional 64-bit - convertible - 1366 x 768 multi-touch screen display led backlight - bluetooth - french keyboard - 34352jf - softwarecity.ca - canada',
 'screen_size': '12.5',
 'model': 'thinkpad x230 34352jf'}

In [31]:
with open('to_dedupe_dict.pkl', 'wb') as f:
    pickle.dump(to_dedupe_dict, f)
with open('to_dedupe_dict.pkl', 'rb') as f:
    to_dedupe_dict = pickle.load(f)

In [32]:
# docs for this are here: 
fields = [{'field' : 'brand', 'type' : 'Categorical', 'categories' : extra_brands},
          
          {'field' : 'cpu_brand', 'type': 'Categorical', 'categories' : ['amd', 'intel']}, 
          
#           {'field' : 'cpu_model', 'type': 'String', 'has_missing' : True},
          
          {'field' : 'cpu_type', 'type': 'Exact', 'has_missing' : False},
          
          {'field' : 'ram_capacity', 'type': 'Price', 'has_missing' : False},
          
          {'field' : 'hdd_capacity', 'type': 'Price', 'has_missing' : False},
          
          {'field' : 'ssd_capacity', 'type': 'Price', 'has_missing' : False},
          
          {'field' : 'title', 'type': 'Text', 'has_missing' : False},
          
          {'field' : 'screen_size', 'type': 'Categorical', 'has_missing' : False, 'categories' : screen_sizes},
          
          {'field' : 'model', 'type': 'String', 'has_missing' : True},
          
         ]

In [46]:
# There is a bug later on that requires num_cores to be 1, but we can make use of
# multi-threaded processes in the meantime
deduper = dedupe.Dedupe(fields, num_cores=24)


In [34]:
y2 = pd.read_csv('../data/sigmod/Y2.csv')
y3 = pd.read_csv('../data/sigmod/Y3.csv')
# y = y3.append(y2)
y = y2
len(y)

58653

In [35]:
training_data = {'match': [], 'distinct': []}

In [36]:
match = y[y.label == 1].to_dict(orient='row')
distinct = y[y.label == 0].to_dict(orient='row')

d:\coding projects\sigmod\venv\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(
d:\coding projects\sigmod\venv\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [37]:
for m in match:
    training_data['match'].append( ( to_dedupe_dict[m['left_instance_id']], to_dedupe_dict[m['right_instance_id']] ) )

In [38]:
len(training_data['match'])

2152

In [39]:
for d in distinct:
    training_data['distinct'].append( ( to_dedupe_dict[d['left_instance_id']], to_dedupe_dict[d['right_instance_id']] ) )

In [40]:
len(training_data['distinct'])

56501

In [41]:
# training_data['match'].extend(my_own_annotation['match'])

In [42]:
# training_data['distinct'].extend(my_own_annotation['distinct'])

In [43]:
with open('y_laptop.json', 'w') as fout:
    json.dump(training_data, fout)

In [44]:
training_file = 'y_laptop.json'
with open(training_file) as tf:
    deduper.prepare_training(to_dedupe_dict, training_file=tf, sample_size=1500, blocked_proportion=0.9)

INFO:dedupe.api:reading training from file
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (twoGramFingerprint, title)
d:\coding projects\sigmod\venv\lib\site-packages\rlr\lr.py:35: UserWarning: The line-search routine reaches the maximum number of evaluations.
  final_betas = opt.minimize(loss,
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, screen_size)
INFO:dedupe.training:SimplePredicate: (sortedAcronym, title)


In [45]:
print('starting active labeling...')
dedupe.console_label(deduper)

brand : lenovo
cpu_brand : intel
cpu_type : i5
ram_capacity : 4
hdd_capacity : 0
ssd_capacity : 240
title : lenovo thinkpad x1 carbon touch 3460 - 14 - core i5 3427u - windows 8 pro 64-bit - 4 gb ram - 240 gb ssd - vology
screen_size : 14
model : thinkpad x1

brand : lenovo
cpu_brand : intel
cpu_type : i5
ram_capacity : 4
hdd_capacity : 0
ssd_capacity : 240
title : lenovo thinkpad x1 carbon touch 3460 - 14 - core i5 3427u - windows 8 pro 64-bit - 4 gb ram - 240 gb ssd - vology
screen_size : 14
model : thinkpad x1

2152/10 positive, 56501/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished
(y)es / (n)o / (u)nsure / (f)inished
(y)es / (n)o / (u)nsure / (f)inished
(y)es / (n)o / (u)nsure / (f)inished
(y)es / (n)o / (u)nsure / (f)inished
(y)es / (n)o / (u)nsure / (f)inished
brand : lenovo
cpu_brand : intel
cpu_type : i7
ram_capacity : 8
hdd_capacity : 0
ssd_capacity : 180
title : lenovo thinkpad x1 carbon 3444 - 14 - core i7 3667u - windows 8 pro 64-b

starting active labeling...


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
with open('y_laptop_augmented.json', 'w') as fout:
    json.dump(deduper.training_pairs, fout)

In [ ]:
deduper.train(recall=0.9)

In [ ]:
training_file = 'trained_model_laptops.json'
settings_file = 'trained_model_laptops_settings.json'
with open(training_file, 'w') as tf:
    deduper.write_training(tf)
with open(settings_file, 'wb') as sf:
    deduper.write_settings(sf)

In [ ]:
clustered_dupes = deduper.partition(to_dedupe_dict, 0.5)

print('# duplicate sets', len(clustered_dupes))

In [ ]:
deduper.predicates

In [ ]:
clustered_dupes

In [ ]:
cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

In [ ]:
cluster_membership

In [ ]:
def laptop_test(x):
    x.set_index('instance_id',  inplace=True, drop=False)
    x = preprocess_laptop_dataset(x.copy(deep=True))
    
    to_dedupe = x[[ 'instance_id',
    'brand',
    'cpu_brand',
    'cpu_type',
    'ram_capacity',
    'hdd_capacity', 
    'ssd_capacity',
    'title',
    'screen_size',
    'model']].copy()
    
    with open('trained_model_laptops_settings.json', 'rb') as fin:
        dr = dedupe.StaticDedupe(fin)
    to_dedupe_dict = to_dedupe.to_dict(orient = 'index')
    
    clustered_dupes = dr.partition(to_dedupe_dict, 0.5)

    print('# duplicate sets', len(clustered_dupes))
    
    res = []
    for el in clustered_dupes:
        for i in range(len(el[0])):
            for j in range(i+1, len(el[0])):
                res.append((el[0][i], el[0][j]))
    res_df =pd.DataFrame(res)            
#     res_df.columns = ['left_instance_id', 'right_instance_id']
    res_df.columns = ['left_instance_id', 'right_instance_id']
    return res_df

In [ ]:
x2 = pd.read_csv("../data/sigmod/X2.csv")
res = laptop_test(x2)

In [ ]:
x3 = pd.read_csv("../data/sigmod/X3.csv")
res = laptop_test(x3)

In [ ]:
len(res)

In [ ]:
res

In [ ]:
res.to_csv("output_x2.csv", index=False)

In [ ]:
deduper.predicates

In [ ]:
deduper.data_model.predicates